In [1]:
import logging
import tensorflow as tf

from src.common.logs import setup_logging
from src.models_service.models_service import TensorflowModelTuningService as TFMTS
from env import Env


logger = setup_logging(logger_name=__name__,
                        console_level=logging.INFO, 
                        log_file_level=logging.INFO)

config = {
    'AV': {
        'key': '',
        'ticker': 'VGT',
        'outputsize': 'full',
        'key_adjusted_close': 'Adj Close',
        'key_volume': 'Volume',
    },
    'data': {
        'test_size': 0.07,
    }, 
    'model': {
        'name': 'LSTM', 
        'window': [20, 40, 60, 120],
        'shuffle_buffer_size' : [0.2, 0.5, 1], # In % from dataset len. https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle
        'batch_size' : [64, 128],
        'epochs' : [5, 6, 7, 8, 9, 10, 12, 15, 20, 25],
# ----------------------------------
        'optimizer': 'adam',
        'loss': 'huber_loss',
        'metrics': ['mae', 'mse'],
    },
    'plots': {
        'loss_zoom': 0.9,
        'show': False,
    },
}

2023-10-20 00:02:08.761699: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-20 00:02:08.761750: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-20 00:02:08.761775: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-20 00:02:08.768751: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Grig Search

In [2]:
model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(None, 2)), # where 2 is the number of features
        tf.keras.layers.LSTM(32),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(1),
        ]
    )

# model.save_weights(f'{Env.PROJECT_ROOT}/models_trained/keep/my_model_weights.h5')
grid_search_service = TFMTS(model, config)
grid_search_service.grid_search(logger)

2023-10-20 00:02:11.646011: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-20 00:02:11.649977: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-20 00:02:11.650017: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-20 00:02:11.651336: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-20 00:02:11.651366: I tensorflow/compile

Epoch 1/5


2023-10-20 00:02:17.710755: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2023-10-20 00:02:19.017610: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f62381e3090 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-20 00:02:19.017653: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2023-10-20 00:02:19.023036: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-20 00:02:19.106820: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


72/72 [==============================] - 7s 16ms/step - loss: 83.5727 - mae: 84.0723 - mse: 13394.9082
Epoch 2/5
 1/72 [..............................] - ETA: 9s - loss: 78.6756 - mae: 79.1756 - mse: 6291.3281

2023-10-20 00:02:20.761929: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 13916981899345554331
2023-10-20 00:02:20.762005: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3523250572394225229


...

288/288 [==============================] - 3s 10ms/step


2023-10-20 00:55:47.842210: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 3523250572394225229
2023-10-20 00:55:48 - src.common.plots - INFO - Plot "Pred-VGT_LSTM_W60_SBS1000_B64_E7_P42113_2023_10_20__00_55.png" saved to "/workspaces/P7-Trading-Bot/figures/Pred-VGT_LSTM_W60_SBS1000_B64_E7_P42113_2023_10_20__00_55.png"
2023-10-20 00:55:48 - src.models_service.errors - INFO - Test RMSE: $ 267.31
2023-10-20 00:55:48 - src.models_service.errors - INFO - Test MAE : $ 264.463
2023-10-20 00:55:48 - src.models_service.errors - INFO - Test MAPE:   0.732
2023-10-20 00:55:48 - src.models_service.errors - INFO - Test MASE:   55.449
2023-10-20 00:55:48 - src.models_service.errors - INFO - Errors saved to for VGT_LSTM_W60_SBS1000_B64_E7_P42113_2023_10_20__00_55 model to "logs/models_table.csv" file.
2023-10-20 00:55:48 - src.data.get_data - INFO - Loaded "../P7-Trading-Bot/data/03_processed/daily_full". Number data points 4965. From "2004-01-30 0

/workspaces/P7-Trading-Bot/models_trained


2023-10-20 00:55:48 - __main__ - INFO - x.shape: (64, 60, 2), y.shape: (64,)
2023-10-20 00:55:48 - __main__ - INFO - Full Dataset shape: (None, None, 2)
2023-10-20 00:55:48 - __main__ - INFO - Input for the model: (None, 2)
2023-10-20 00:55:48 - __main__ - INFO - ==============================================================================================
2023-10-20 00:55:48 - __main__ - INFO - =================================== MODEL CONFIG AND SETUP ===================================
2023-10-20 00:55:48 - __main__ - INFO - ==============================================================================================
2023-10-20 00:55:48 - __main__ - INFO - AV_key: , AV_ticker: VGT, AV_outputsize: full, AV_key_adjusted_close: Adj Close, AV_key_volume: Volume, 
2023-10-20 00:55:48 - __main__ - INFO - data_test_size: 0.07, 
2023-10-20 00:55:48 - __main__ - INFO - ----------------------------------------------------------------------
2023-10-20 00:55:48 - __main__ - INFO - model_name: 

Epoch 1/8
72/72 [==============================] - 6s 33ms/step - loss: 82.6202 - mae: 83.1197 - mse: 13030.1035
Epoch 2/8
72/72 [==============================] - 2s 32ms/step - loss: 76.6732 - mae: 77.1718 - mse: 12324.4980
Epoch 3/8
72/72 [==============================] - 2s 32ms/step - loss: 73.9440 - mae: 74.4412 - mse: 12102.0908
Epoch 4/8
72/72 [==============================] - 2s 30ms/step - loss: 72.5319 - mae: 73.0300 - mse: 11959.4688
Epoch 5/8
55/72 [=====================>........] - ETA: 0s - loss: 49.7595 - mae: 50.2572 - mse: 4820.5752